# Convolutional Auto Encoder
Convolutional auto encoder was first introdueced in the below paper.
Stacked Convolutional Auto-Encoders for Hierarchical Feature Extraction, Jonathan Masci, U. Meier, D. Ciresan, J. Schmidhuber
https://people.idsia.ch/~ciresan/data/icann2011.pdf


In [1]:
import os

import torch
import torch.nn as nn
from torchvision import datasets

import torchvision.transforms as transforms
from torch.utils.data import DataLoader


from torchvision.utils import save_image

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
batch_size = 100

input_dim = 3
hidden_dim = 128
n_embeddings= 768
output_dim = 3

lr = 1e-3
weight_decay =1e-5

n_epochs = 10

In [3]:
data_path = '../data'

In [4]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize([0.5], [0.5])])

loader_kwargs = {'num_workers': os.cpu_count()//2, 'pin_memory': True} 

train_data = datasets.CIFAR10(root=data_path, train=True, download=True, transform=transform)
test_data = datasets.CIFAR10(root=data_path, train=False, download=True, transform=transform)

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, **loader_kwargs)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False, **loader_kwargs)


Files already downloaded and verified
Files already downloaded and verified


In [5]:
class AutoEncoder(nn.Module):

    def __init__(self):
        super(AutoEncoder,self).__init__()
        
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 6, kernel_size=5),
            nn.LeakyReLU(True),
            nn.Conv2d(6,16,kernel_size=5),
            nn.LeakyReLU(True))

        self.decoder = nn.Sequential(             
            nn.ConvTranspose2d(16,6,kernel_size=5),
            nn.LeakyReLU(True),
            nn.ConvTranspose2d(6,3,kernel_size=5),
            nn.LeakyReLU(True),
            nn.Sigmoid())

    def forward(self,x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

model = AutoEncoder().to(device)

In [6]:
criterion = nn.MSELoss()
optimizer = torch.optim.AdamW(model.parameters(), lr = lr)

In [7]:
saved_dir = 'cvae_images'
os.makedirs(saved_dir, exist_ok= True)

In [8]:
loss_list = []
for epoch in range(n_epochs):
    for batch_idx, data in enumerate(train_loader):
        img, label = data
        img = img.to(device)

        output = model(img)
        loss = criterion(output, img)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        loss_list.append(loss.item())

    
    # ===== save images and print logs =====
    save_image(output.data[:25], f"{saved_dir}/{epoch+1}.png", nrow=5, normalize=True)
    print(f'epoch: {epoch+1}/{n_epochs}, loss: {loss.item() :.4f}')

epoch: 1/10, loss:0.1576
epoch: 2/10, loss:0.1584
epoch: 3/10, loss:0.1674
epoch: 4/10, loss:0.1446
epoch: 5/10, loss:0.1633
epoch: 6/10, loss:0.1348
epoch: 7/10, loss:0.1536
epoch: 8/10, loss:0.1598
epoch: 9/10, loss:0.1348
epoch: 10/10, loss:0.1490
